In [1]:
# Suzan Iloglu, May 21,2020
# Import packages
import csv
import gurobipy as gp
from itertools import product
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import time
import requests
import io
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.options.display.max_columns =200
from IPython.display import Image


# MAPPING THE NEW POLITICS OF CARE: COMMUNITY HEALTH WORKERS
The project presents multiple options for how individual workers in such a Community Health Corps might be distributed within each state. It shows that what you choose to prioritize greatly impacts where care would be sent. We can define communities in greatest need in many ways: we can think about our current crisis and send people to where the COVID19 pandemic rages most fiercely; we can think of long term measures of social and economic inequality embedded in metrics like the Centers for Disease Control and Prevention’s Social Vulnerability Index; we can focus on the places with too many people dying too young and use the County Health Rankings Years-of-Potential-Life-Lost measure; we can think of joblessness and how the pandemic has thrown many into unemployment and target our resources in this way. 

The followings are our options to choose to define vulnerability:


- SOCIAL VULNERABILITY INDEX
- MEDICAID 
- UNEMPLOYMENT
- YEARS OF POTENTIAL LIFE LOST
- TOTAL COVID CASES
- COVID CASES BY POPULATION
- COVID DEATHS BY POPULATION

We will start with Social Vulnerability Index (SVI) from CDC website.

### I. Importing SVI data which includes the variables for calculating county SVI for each state
The CDC uses both a USA-wide and a state by state SVI scores. For our project given that funding is likely going to be managed at a state level, using a state by state SVI scores makes the most sense and will be most sensitive to regional socioeconomic differences. Even though the CDC SVI scores are calculated using percentile rankings, the data sets include raw data estimates for each variables. The following table shows the variablaes used in the method of calculating SVI scores. 




      American Community Survey (ACS), 2014-2018 (5-year) data for the following estimates:
<img src="Data/img/SVI_comp.png" width="500">


Note: Full documentation for 2018 data is available <a href="https://svi.cdc.gov/data-and-tools-download.html">here</a> 
This part of the code shows preliminary mapping of <a href = "https://svi.cdc.gov/">the CDC's Social Vulnerability Index</a>.

Later in the notebook, we will provide the formula to create the SVI value we use in our project. First, we import the data for the US mainland and Puerto Rico.

In [2]:
## import svi data downloaded from CDC website as cited above

## 48 state SVI scores by county
svi_counties_mainland = gpd.read_file("Data/SVI2018_US_COUNTY/SVI2018_US_county.shp")

## Puerto Rico SVI scores by county
svi_counties_puerto_rico = gpd.read_file("Data/PuertoRico_COUNTY/SVI2018_PuertoRico_county.shp")

## Merge 48 states and Puerto Rico SVI 
svi_counties = pd.concat([svi_counties_mainland,svi_counties_puerto_rico ], sort = False)


In [3]:
## Replacing -999 values with 0 for calculations
svi_county = svi_counties.fillna(0)
svi_county  = svi_county.replace(-999, 0)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)

In [4]:
## Create the list for State
State = svi_county.STATE.unique().tolist()

In [5]:
# Create a seperate dictionary for the variables to calculate SVI

# Persons below poverty estimate, 2014-2018 ACS
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Civilian (age 16+) unemployed estimate, 2014-2018 ACS
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.E_UNEMP))

# Per capita income estimate, 2014-2018 ACS
E_PCI = dict(zip(svi_county.FIPS, svi_county.E_PCI))

# Persons (age 25+) with no high school diploma estimate, 2014-2018 ACS
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.E_NOHSDP))

# Persons aged 65 and older estimate
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.E_AGE65))

# Persons aged 17 and younger estimate
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.E_AGE17))

# Population with a disability estimate
E_DISABL = dict(zip(svi_county.FIPS, svi_county.E_DISABL))

# Single parent households with children under 18 estimate
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.E_SNGPNT))

# Minority (all persons except white, nonHispanic) estimate, 2014-2018 ACS
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.E_MINRTY))

# Persons (age 5+) who speak English "less than well" estimate, 2014-2018 ACS
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.E_LIMENG))

# Housing in structures with 10 or more units estimate, 2014-2018 ACS
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.E_MUNIT))

# Mobile homes estimate MOE, 2014-2018 ACS
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.E_MOBILE))

# At household level (occupied housing units), more people than rooms estimate, 2014-2018 ACS
E_CROWD = dict(zip(svi_county.FIPS, svi_county.E_CROWD))

# Households with no vehicle available estimate, 2014-2018 ACS
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.E_NOVEH))

# Persons in institutionalized group quarters estimate, 2014-2018 ACS
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.E_GROUPQ))

# Percentage of persons below poverty estimate
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Medicaid 
Medicaid is a means-tested health insurance program for low-income children, pregnant women, adults, seniors, and people with disabilities. Medicaid is jointly funded by federal and state governments and managed by states within federal standards and a wide range of state options. <a href="https://data.medicaid.gov/Enrollment/State-Medicaid-and-CHIP-Applications-Eligibility-D/n5ce-jxme"> Data Source for Medicaid Enrollment </a> 

In [6]:

import sodapy
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.medicaid.gov", None)


# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("83yt-67it", limit=4000)


# Read the medicaid demand data
#df_mm = pd.read_csv("Data/2020_06_Preliminary_applications__eligibility_determinations__and_enrollment_data.csv")

# Convert to pandas DataFrame
df_mm = pd.DataFrame.from_records(results)
df_mm.head(5)
df_mm.columns


Index(['applications_for_financial_assistance_submitted_to_the_state_based_marketplace',
       'applications_for_financial_assistance_submitted_to_the_state_based_marketplace_footnotes',
       'final_report', 'geocoded_column',
       'individuals_determined_eligible_for_chip_at_application',
       'individuals_determined_eligible_for_chip_at_application_footnotes',
       'individuals_determined_eligible_for_medicaid_at_application',
       'individuals_determined_eligible_for_medicaid_at_application_footnotes',
       'latitude', 'longitude', 'medicaid_and_chip_child_enrollment',
       'medicaid_and_chip_child_enrollment_footnotes',
       'new_applications_submitted_to_medicaid_and_chip_agencies',
       'new_applications_submitted_to_medicaid_and_chip_agencies_footnotes',
       'preliminary_updated', 'report_date', 'state_abbreviation',
       'state_expanded_medicaid', 'state_name',
       'total_applications_for_financial_assistance_submitted_at_state_level',
       'total_a

In [7]:
df_mm['State Name'] = df_mm['state_name'].str.upper() 

In [8]:
Medicaid_state = dict(zip(df_mm['State Name'], df_mm['total_medicaid_and_chip_enrollment']))
Medicaid_state['PUERTO RICO'] = 1622194
print (Medicaid_state)

{'ALABAMA': '957116', 'ALASKA': '231145', 'ARIZONA': '1839932', 'ARKANSAS': '830467', 'CALIFORNIA': '11847711', 'COLORADO': '1337805', 'CONNECTICUT': '874974', 'DELAWARE': '239009', 'DISTRICT OF COLUMBIA': '248591', 'FLORIDA': '3892552', 'GEORGIA': '1928703', 'HAWAII': '351337', 'IDAHO': '340742', 'ILLINOIS': '2987496', 'INDIANA': '1602976', 'IOWA': '699741', 'KANSAS': '401103', 'KENTUCKY': '1416013', 'LOUISIANA': '1585024', 'MAINE': '232455', 'MARYLAND': '1372695', 'MASSACHUSETTS': '1616404', 'MICHIGAN': '2439425', 'MINNESOTA': '1085778', 'MISSISSIPPI': '632427', 'MISSOURI': '923641', 'MONTANA': '257006', 'NEBRASKA': '254159', 'NEVADA': '685073', 'NEW HAMPSHIRE': '193436', 'NEW JERSEY': '1759653', 'NEW MEXICO': '772102', 'NEW YORK': '6263164', 'NORTH CAROLINA': '1851558', 'NORTH DAKOTA': '96757', 'OHIO': '2788134', 'OKLAHOMA': '797220', 'OREGON': '1053931', 'PENNSYLVANIA': '3069309', 'RHODE ISLAND': '305208', 'SOUTH CAROLINA': '1048276', 'SOUTH DAKOTA': '114059', 'TENNESSEE': '1489536

In [9]:
df_mmm = pd.read_csv("Data/ACSST5Y2019.S2704_data_with_overlays_2020-12-11T225641.csv", header=[1])
df_mmm.head(1)
#df_mmm.dtypes

,id,Geographic Area Name,Estimate!!Total!!Civilian noninstitutionalized population,Margin of Error!!Total!!Civilian noninstitutionalized population,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,"Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)","Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)",Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALON

In [10]:
df_mmm['FIPS'] = df_mmm.id.astype(str).str[9:]
df_mmm.head(5)
df_mmm.FIPS.astype(int)
df_mmm['FIPS'] = pd.to_numeric(df_mmm['FIPS'])

In [11]:
# 2018 ACI data
# ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!Medicaid/means-tested public coverage alone or in combination"]))

#2019 ACI data
ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination"]))




In [12]:

#df_m = pd.read_csv("Data/Medicaid_Demand.csv")

# Unemployment 
The unemployment rate is calculated by the U.S. Bureau of Labor Statistics as the percentage of the civilian labor force who are without jobs and have actively sought work within the past four weeks. <a href="https://www.bls.gov/lau/laufaq.htm#Q01"> Data Source for Unemployment  </a> 

In [13]:
from io import StringIO
import datetime 
from datetime import date
from dateutil.relativedelta import relativedelta

back = date.today() + relativedelta(months= -3)
three_months_ago = back.strftime('%b-%y') 


url = 'https://www.bls.gov/web/metro/laucntycur14.txt'
s = requests.get(url).text



df_unemp = pd.read_csv(StringIO(s), sep='|',  skiprows=7, skipfooter=6, engine='python', names = ['LAUS Area Code', 'FIPS State', 'FIPS County', 'Area Title', 'Period', 'Civilian Labor Force','Employed','Unemployed_Level','Unemployed_Rate'])

df_unemp['Period'] = df_unemp['Period'].astype(str)

df_unemp['FIPS'] = df_unemp['LAUS Area Code'].str[3:8]

df_unemp = df_unemp[df_unemp['Period'].str.contains(str(three_months_ago))]

df_unemp.head(5)
#df_unemp.dtypes

,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
38627,CN0100100000000,1,1,"Autauga County, AL",Oct-20,"25,237","24,120","1,117",4.4,01001
38628,CN0100300000000,1,3,"Baldwin County, AL",Oct-20,"95,828","91,145","4,683",4.9,01003
38629,CN0100500000000,1,5,"Barbour County, AL",Oct-20,"9,633","8,887",746,7.7,01005
38630,CN0100700000000,1,7,"Bibb County, AL",Oct-20,"8,607","8,116",491,5.7,01007
38631,CN0100900000000,1,9,"Blount County, AL",Oct-20,"24,646","23,830",816,3.3,01009


In [14]:
#df_unemp[[ 'FIPS', 'Area Title', 'Period','Unemployed_Level','Unemployed_Rate']].to_csv('Data/County_employment.csv', index=False)


In [15]:


df_unemp.replace({'-', 0})

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.replace(',', '')

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].astype(str)

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.strip()

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].str.strip()

df_unemp['FIPS'] = pd.to_numeric(df_unemp['FIPS'])

df_unemp['Unemployed_Level'] = pd.to_numeric(df_unemp['Unemployed_Level'])

df_unemp['Unemployed_Rate'] = pd.to_numeric(df_unemp['Unemployed_Rate'])




# Fill NA with 0
#df_unemp = df_unemp.fillna(0)
df_unemp.tail(5)
#df_unemp.dtypes


,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
41841,CN7214500000000,72,145,"Vega Baja Municipio, PR",Oct-20,"12,548","11,152",1396,11.1,72145
41842,CN7214700000000,72,147,"Vieques Municipio, PR",Oct-20,"2,389","2,135",254,10.6,72147
41843,CN7214900000000,72,149,"Villalba Municipio, PR",Oct-20,"6,563","5,929",634,9.7,72149
41844,CN7215100000000,72,151,"Yabucoa Municipio, PR",Oct-20,"7,959","7,172",787,9.9,72151
41845,CN7215300000000,72,153,"Yauco Municipio, PR",Oct-20,"9,190","8,260",930,10.1,72153


In [16]:
 
## Read the Unemployment data using cvs data 
#df_unemp = pd.read_csv("Data/Unemployment.csv")

# Fill NA with 0
#df_unemp = df_unemp.fillna(0)

#df_unemp.head(5)

# Years of Potential Life Lost (YPLL)

Years of Potential Life Lost (YPLL) measures the rate of premature deaths by region. YPLL is calculated as the sum of the estimated number of years that individuals would have lived if they had not died before the age of 75 per 100,000 people. <a href="https://www.countyhealthrankings.org/sites/default/files/media/document/2020%20County%20Health%20Rankings%20Data%20-%20v2.xlsx"> Data Source for YPLL.  </a> More information about YPLL can be dounf in this <a href="https://www.countyhealthrankings.org/explore-health-rankings/measures-data-sources/county-health-rankings-model/health-outcomes/length-of-life/premature-death-ypll"> link. </a> 


In [17]:
# Read the YPLL data
df_y = pd.read_csv("Data/YPLL.csv")

# Fill NA with the mean of the data
df_y = df_y.fillna(df_y.mean())


# Population

In [18]:
# Read the Population data
df_pop = pd.read_csv("Data/County_pop_2019.csv")

# Fill NA with 0
df_pop = df_pop.fillna(0)


In [19]:
# Create a dictionary for the county and population
population_county = df_pop.set_index('FIPS')['pop'].to_dict()

# Create a dictionary for the county and YPLL
YPLL = dict(zip(df_y.FIPS, df_y.YPLL))

# Create a dictionary for the county and Unemployment
Unemployment = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Level))


# Create a dictionary for the county and Community Health Workers (CHW) demand
# Note that we assume a CHW can serve 55 Medicaid patient so the demand for CHW will be

#Medicaid_demand = dict(zip(df_m.FIPS, df_m.Med_Demand))


In [20]:
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

# COVID-19 Cases & COVID-19 Cases per Capita

What are COVID-19 Cases and COVID-19 Cases per Capita?

COVID-19 cases is an absolute metric of the total number of COVID-19 cases in a county over the last fourteen days.  COVID-19 cases per 100,000 is a relative metric calculated by dividing the number of COVID-19 cases by the estimated county population and multiplying by 100,000.  Cases include both confirmed cases, based on viral testing, and probable cases, based on specific criteria for symptoms and epidemiological exposure. We use NY Times Covid data. 


In [21]:
#### Data with the most recent date in NY Times dataset:

today = time.strftime('%Y-%m-%d')
covid_data_update_date = today#'2020-07-21'#today #or enter a specific date such as '2020-07-06'


In [22]:
## 14 day period defined
data_date_dt = pd.to_datetime(covid_data_update_date,infer_datetime_format = True)

N = 14

date_N_days_ago = data_date_dt - timedelta(days = N)

date_N1_days_ago = data_date_dt - timedelta(days = N+1)

In [23]:

# URL for mainland US data
url = "http://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
s = requests.get(url).content
covid = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [24]:
covid.tail(50)

,date,county,state,fips,cases,deaths
907488,2021-01-07,Ozaukee,Wisconsin,55089.0,7393,62.0
907489,2021-01-07,Pepin,Wisconsin,55091.0,716,6.0
907490,2021-01-07,Pierce,Wisconsin,55093.0,3739,35.0
907491,2021-01-07,Polk,Wisconsin,55095.0,3227,29.0
907492,2021-01-07,Portage,Wisconsin,55097.0,6134,56.0
907493,2021-01-07,Price,Wisconsin,55099.0,1043,5.0
907494,2021-01-07,Racine,Wisconsin,55101.0,20502,278.0
907495,2021-01-07,Richland,Wisconsin,55103.0,1180,13.0
907496,2021-01-07,Rock,Wisconsin,55105.0,13634,128.0
907497,2021-01-07,Rusk,Wisconsin,55107.0,1177,14.0


Note: Since NY data is seperately available, we first read the NY data for all 5 different borough then combine with the rest of US data.

In [25]:
# URL for NY
url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/data-by-day.csv"
#"https://raw.githubusercontent.com/nychealth/coronavirus-data/master/data-by-day.csv"

ny = requests.get(url).content
covid_ny = pd.read_csv(io.StringIO(ny.decode('utf-8')))


covid_ny.tail(5)

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,ALL_DEATH_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_HOSPITALIZED_COUNT,BX_DEATH_COUNT,BX_PROBABLE_DEATH_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BX_HOSPITALIZED_COUNT_7DAY_AVG,BX_DEATH_COUNT_7DAY_AVG,BX_ALL_DEATH_COUNT_7DAY_AVG,BK_CASE_COUNT,BK_PROBABLE_CASE_COUNT,BK_HOSPITALIZED_COUNT,BK_DEATH_COUNT,BK_PROBABLE_DEATH_COUNT,BK_CASE_COUNT_7DAY_AVG,BK_ALL_CASE_COUNT_7DAY_AVG,BK_HOSPITALIZED_COUNT_7DAY_AVG,BK_DEATH_COUNT_7DAY_AVG,BK_ALL_DEATH_COUNT_7DAY_AVG,MN_CASE_COUNT,MN_PROBABLE_CASE_COUNT,MN_HOSPITALIZED_COUNT,MN_DEATH_COUNT,MN_PROBABLE_DEATH_COUNT,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,MN_HOSPITALIZED_COUNT_7DAY_AVG,MN_DEATH_COUNT_7DAY_AVG,MN_ALL_DEATH_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_HOSPITALIZED_COUNT,QN_DEATH_COUNT,QN_PROBABLE_DEATH_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,QN_HOSPITALIZED_COUNT_7DAY_AVG,QN_DEATH_COUNT_7DAY_AVG,QN_ALL_DEATH_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_PROBABLE_DEATH_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,SI_ALL_DEATH_COUNT_7DAY_AVG,INCOMPLETE
307,01/01/2021,1218,727,267,34,5,3663,4663,264,38,43,187,124,66,9,1,666,839,58,6,6,379,169,68,15,1,1063,1323,77,13,15,150,119,39,2,1,446,585,43,5,6,387,236,70,5,0,1093,1430,69,8,9,115,79,24,3,0,395,486,18,5,5,6000
308,01/02/2021,3984,966,267,49,7,3756,4750,266,40,45,678,164,48,12,1,689,861,58,7,7,1048,229,74,16,2,1071,1325,75,14,16,498,171,50,2,0,458,604,44,5,5,1271,291,72,11,0,1128,1459,70,8,9,489,111,23,8,0,410,502,19,6,6,6000
309,01/03/2021,3132,878,267,30,6,3803,4779,272,38,44,471,131,48,7,1,689,855,59,7,8,951,235,81,9,0,1080,1332,75,14,15,367,140,38,5,0,468,613,45,4,4,975,289,83,4,1,1147,1469,73,8,9,368,83,17,5,0,419,510,19,6,6,6000
310,01/04/2021,5281,1269,317,43,15,3810,4785,274,39,46,1044,208,64,7,1,699,863,59,7,8,1509,316,92,17,2,1075,1319,76,14,16,776,230,52,3,0,483,637,44,4,4,1509,374,84,10,0,1151,1465,74,8,8,442,140,25,6,0,402,501,20,5,6,6000
311,01/05/2021,3197,1267,188,40,21,3550,4552,262,38,47,638,223,52,10,0,653,820,57,7,8,905,347,38,14,0,1006,1259,72,15,16,566,204,24,6,1,481,643,41,4,4,876,382,59,8,0,1059,1377,72,8,8,211,110,15,2,0,351,452,21,4,5,6000


In [26]:
Kings = covid_ny[['date_of_interest', 'BK_CASE_COUNT', 'BK_DEATH_COUNT']]
Kings.rename(columns = {'BK_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BK_DEATH_COUNT': 'deaths'} , inplace=True)
Kings['county'] = 'Kings'
Kings['state'] = 'New York'
Kings['fips'] = 36047.0
#Kings.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [27]:
Bronx = covid_ny[['date_of_interest', 'BX_CASE_COUNT', 'BX_DEATH_COUNT']]
Bronx.rename(columns = {'BX_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BX_DEATH_COUNT': 'deaths'} , inplace=True)
Bronx['state'] = 'New York'
Bronx['county'] = 'Bronx'
Bronx['fips'] = 36005.0
#Bronx.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [28]:
Manhattan = covid_ny[['date_of_interest', 'MN_CASE_COUNT', 'MN_DEATH_COUNT']]
Manhattan.rename(columns = {'MN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'MN_DEATH_COUNT': 'deaths'} , inplace=True)
Manhattan['state'] = 'New York'
Manhattan['county'] = 'Manhattan'
Manhattan['fips'] = 36061.0
#Manhattan.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [29]:
Queens = covid_ny[['date_of_interest', 'QN_CASE_COUNT', 'QN_DEATH_COUNT']]
Queens.rename(columns = {'QN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'QN_DEATH_COUNT': 'deaths'} , inplace=True)
Queens['state'] = 'New York'
Queens['county'] = 'Queens'
Queens['fips'] = 36081.0
#Queens.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [30]:
Richmond = covid_ny[['date_of_interest', 'SI_CASE_COUNT', 'SI_DEATH_COUNT']]
Richmond.rename(columns = {'SI_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'SI_DEATH_COUNT': 'deaths'} , inplace=True)
Richmond['state'] = 'New York'
Richmond['county'] = 'Richmond'
Richmond['fips'] = 36085.0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [31]:
covid.head(5)


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [32]:
Bronx.tail(5)

,date,cases,deaths,state,county,fips
307,01/01/2021,187,9,New York,Bronx,36005.0
308,01/02/2021,678,12,New York,Bronx,36005.0
309,01/03/2021,471,7,New York,Bronx,36005.0
310,01/04/2021,1044,7,New York,Bronx,36005.0
311,01/05/2021,638,10,New York,Bronx,36005.0


In [33]:
#Kings['deaths'] = Kings.groupby(by=['fips'])['deaths_d'].sum()
#Bronx['deaths'] = Bronx.groupby(by=['fips'])['deaths_d'].sum()
#Manhattan['deaths'] = Manhattan.groupby(by=['fips'])['deaths_d'].sum()
#Queens['deaths'] = Queens.groupby(by=['fips'])['deaths_d'].sum()
#Richmond['deaths'] = Richmond.groupby(by=['fips'])['deaths_d'].sum()

In [34]:
Kings.head(5)

,date,cases,deaths,county,state,fips
0,02/29/2020,0,0,Kings,New York,36047.0
1,03/01/2020,0,0,Kings,New York,36047.0
2,03/02/2020,0,0,Kings,New York,36047.0
3,03/03/2020,0,0,Kings,New York,36047.0
4,03/04/2020,1,0,Kings,New York,36047.0


In [35]:
#Kings = Kings.drop(['deaths_d'], axis=1)
#Bronx = Bronx.drop(['deaths_d'], axis=1)
#Manhattan = Manhattan.drop(['deaths_d'], axis=1)
#Queens = Queens.drop(['deaths_d'], axis=1)
#Richmond = Richmond.drop(['deaths_d'], axis=1)

In [36]:
Kings.tail(5)

,date,cases,deaths,county,state,fips
307,01/01/2021,379,15,Kings,New York,36047.0
308,01/02/2021,1048,16,Kings,New York,36047.0
309,01/03/2021,951,9,Kings,New York,36047.0
310,01/04/2021,1509,17,Kings,New York,36047.0
311,01/05/2021,905,14,Kings,New York,36047.0


In [37]:
covid['dt'] = pd.to_datetime(covid['date'], infer_datetime_format=True)
Kings['dt'] = pd.to_datetime(Kings['date'], infer_datetime_format=True)
Bronx['dt'] = pd.to_datetime(Bronx['date'], infer_datetime_format=True)
Manhattan['dt'] = pd.to_datetime(Manhattan['date'], infer_datetime_format=True)
Queens['dt'] = pd.to_datetime(Queens['date'], infer_datetime_format=True)
Richmond['dt'] = pd.to_datetime(Richmond['date'], infer_datetime_format=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [38]:

Kings_c = Kings[(Kings['dt']>date_N1_days_ago) & (Kings['dt']<= data_date_dt)].copy()
Queens_c = Queens[(Queens['dt']>date_N1_days_ago) & (Queens['dt']<= data_date_dt)].copy()
Bronx_c = Bronx[(Bronx['dt']>date_N1_days_ago) & (Bronx['dt']<= data_date_dt)].copy()
Manhattan_c = Manhattan[(Manhattan['dt']>date_N1_days_ago) & (Manhattan['dt']<= data_date_dt)].copy()
Richmond_c = Richmond[(Richmond['dt']>date_N1_days_ago) & (Richmond['dt']<= data_date_dt)].copy()

In [39]:
#Kings['cases'] = Kings.loc[(Kings['dt']<= data_date_dt)].groupby(['dt'])['d_cases'].sum()
#Kings['cases'] = Kings.apply(lambda x: x[(Kings['dt']<= data_date_dt)]['d_cases'].sum())

In [40]:
Kings.head(50)

,date,cases,deaths,county,state,fips,dt
0,02/29/2020,0,0,Kings,New York,36047.0,2020-02-29
1,03/01/2020,0,0,Kings,New York,36047.0,2020-03-01
2,03/02/2020,0,0,Kings,New York,36047.0,2020-03-02
3,03/03/2020,0,0,Kings,New York,36047.0,2020-03-03
4,03/04/2020,1,0,Kings,New York,36047.0,2020-03-04
5,03/05/2020,3,0,Kings,New York,36047.0,2020-03-05
6,03/06/2020,1,0,Kings,New York,36047.0,2020-03-06
7,03/07/2020,2,0,Kings,New York,36047.0,2020-03-07
8,03/08/2020,5,0,Kings,New York,36047.0,2020-03-08
9,03/09/2020,16,0,Kings,New York,36047.0,2020-03-09


In [41]:
K_c = Kings_c.groupby(['fips']).sum().reset_index()
Q_c = Queens_c.groupby(['fips']).sum().reset_index()
B_c = Bronx_c.groupby(['fips']).sum().reset_index()
M_c = Manhattan_c.groupby(['fips']).sum().reset_index()
R_c = Richmond_c.groupby(['fips']).sum().reset_index()
K_c.head(19)

,fips,cases,deaths
0,36047.0,12180,162


In [42]:
covid_death = covid[(covid['dt'] >= data_date_dt- timedelta(days = 1))]
Kings_death = Kings.groupby(by=['fips'])['deaths'].sum().reset_index()
Bronx_death = Bronx.groupby(by=['fips'])['deaths'].sum().reset_index()
Manhattan_death = Manhattan.groupby(by=['fips'])['deaths'].sum().reset_index()
Queens_death = Queens.groupby(by=['fips'])['deaths'].sum().reset_index()
Richmond_death = Richmond.groupby(by=['fips'])['deaths'].sum().reset_index()

In [43]:
# We merge the NY data with the rest of the US data
covid = pd.concat([covid, Kings, Bronx, Manhattan, Queens, Richmond], sort = False)


In [44]:
Queens.tail(14)

,date,cases,deaths,state,county,fips,dt
298,12/23/2020,1159,7,New York,Queens,36081.0,2020-12-23
299,12/24/2020,852,15,New York,Queens,36081.0,2020-12-24
300,12/25/2020,281,7,New York,Queens,36081.0,2020-12-25
301,12/26/2020,1022,11,New York,Queens,36081.0,2020-12-26
302,12/27/2020,847,5,New York,Queens,36081.0,2020-12-27
303,12/28/2020,1480,11,New York,Queens,36081.0,2020-12-28
304,12/29/2020,1519,7,New York,Queens,36081.0,2020-12-29
305,12/30/2020,1428,8,New York,Queens,36081.0,2020-12-30
306,12/31/2020,966,10,New York,Queens,36081.0,2020-12-31
307,01/01/2021,387,5,New York,Queens,36081.0,2021-01-01


In [45]:

covid.tail(50)

,date,county,state,fips,cases,deaths,dt
262,11/17/2020,Richmond,New York,36085.0,190,2.0,2020-11-17
263,11/18/2020,Richmond,New York,36085.0,197,2.0,2020-11-18
264,11/19/2020,Richmond,New York,36085.0,200,0.0,2020-11-19
265,11/20/2020,Richmond,New York,36085.0,222,1.0,2020-11-20
266,11/21/2020,Richmond,New York,36085.0,161,4.0,2020-11-21
267,11/22/2020,Richmond,New York,36085.0,177,1.0,2020-11-22
268,11/23/2020,Richmond,New York,36085.0,289,1.0,2020-11-23
269,11/24/2020,Richmond,New York,36085.0,237,3.0,2020-11-24
270,11/25/2020,Richmond,New York,36085.0,293,4.0,2020-11-25
271,11/26/2020,Richmond,New York,36085.0,90,2.0,2020-11-26


In [46]:
print (data_date_dt)

2021-01-08 00:00:00


In [47]:

covid_death = covid_death[['fips', 'deaths']]
covid_death.head(2)

,fips,deaths
904291,1001.0,50.0
904292,1003.0,171.0


In [48]:
Kings_death.head(2)

,fips,deaths
0,36047.0,6159


In [49]:
# We merge the NY data with the rest of the US data
covid_death = pd.concat([covid_death, Kings_death, Bronx_death, Manhattan_death, Queens_death, Richmond_death], sort = False)

covid_death = covid_death.rename(columns={"deaths": "total_deaths"})


# Create a dictionary for the cumulative COVID deaths in each county
County_covid_death = dict(zip(covid_death.fips, covid_death.total_deaths))

covid_death.tail(250)

,fips,total_deaths
907293,51690.0,27.0
907294,51115.0,4.0
907295,51117.0,39.0
907296,51119.0,12.0
907297,51121.0,45.0
907298,51125.0,4.0
907299,51127.0,5.0
907300,51700.0,72.0
907301,51710.0,113.0
907302,51131.0,31.0


In [50]:
covid_death.shape

(3252, 2)

In [51]:
print (County_covid_death)

{1001.0: 50.0, 1003.0: 171.0, 1005.0: 35.0, 1007.0: 48.0, 1009.0: 72.0, 1011.0: 22.0, 1013.0: 46.0, 1015.0: 158.0, 1017.0: 63.0, 1019.0: 25.0, 1021.0: 54.0, 1023.0: 22.0, 1025.0: 26.0, 1027.0: 34.0, 1029.0: 17.0, 1031.0: 42.0, 1033.0: 62.0, 1035.0: 17.0, 1037.0: 7.0, 1039.0: 43.0, 1041.0: 36.0, 1043.0: 78.0, 1045.0: 64.0, 1047.0: 45.0, 1049.0: 62.0, 1051.0: 85.0, 1053.0: 34.0, 1055.0: 105.0, 1057.0: 21.0, 1059.0: 39.0, 1061.0: 23.0, 1063.0: 20.0, 1065.0: 34.0, 1067.0: 8.0, 1069.0: 70.0, 1071.0: 34.0, 1073.0: 748.0, 1075.0: 16.0, 1077.0: 71.0, 1079.0: 44.0, 1081.0: 79.0, 1083.0: 60.0, 1085.0: 34.0, 1087.0: 35.0, 1089.0: 179.0, 1091.0: 27.0, 1093.0: 46.0, 1095.0: 92.0, 1097.0: 518.0, 1099.0: 12.0, 1101.0: 282.0, 1103.0: 103.0, 1105.0: 19.0, 1107.0: 31.0, 1109.0: 23.0, 1111.0: 31.0, 1113.0: 7.0, 1117.0: 105.0, 1115.0: 74.0, 1119.0: 24.0, 1121.0: 76.0, 1123.0: 100.0, 1125.0: 222.0, 1127.0: 170.0, 1129.0: 26.0, 1131.0: 20.0, 1133.0: 34.0, 2013.0: 0.0, 2016.0: 0.0, 2020.0: 124.0, 2050.0: 12.

In [52]:
## subset last last 15 days
covid_last15 = covid[(covid['dt']>date_N1_days_ago) & (covid['dt']<= data_date_dt)].copy()
covid_last15['dt_time_delta'] = covid_last15['dt']-data_date_dt

In [53]:
## calculate new daily cases

## sort values by county and date
covid_last15.sort_values(by=['fips','dt'],inplace=True)
## remove data with 'unknown' counties
covid_last15 = covid_last15[covid_last15['fips'].notnull()].copy()

## calculate daily difference in number of cases
covid_last15['new_cases'] = covid_last15.groupby('fips')['cases'].transform(lambda x: x.diff())
## set negative new cases to zero, this can occuer due to the disperacy in the data

covid_last15.loc[covid_last15.new_cases < 1e-6, 'new_cases'] = 0
covid_last15.sort_index(inplace = True)

In [54]:
covid_last15[covid_last15.county == 'New Haven'].head(50)

,date,county,state,fips,cases,deaths,dt,dt_time_delta,new_cases
862414,2020-12-25,New Haven,Connecticut,9009.0,44134,1486.0,2020-12-25,-14 days,NaN
865659,2020-12-26,New Haven,Connecticut,9009.0,44134,1486.0,2020-12-26,-13 days,0.0
868904,2020-12-27,New Haven,Connecticut,9009.0,44134,1486.0,2020-12-27,-12 days,0.0
872149,2020-12-28,New Haven,Connecticut,9009.0,46237,1509.0,2020-12-28,-11 days,2103.0
875394,2020-12-29,New Haven,Connecticut,9009.0,46466,1517.0,2020-12-29,-10 days,229.0
878639,2020-12-30,New Haven,Connecticut,9009.0,46892,1525.0,2020-12-30,-9 days,426.0
881884,2020-12-31,New Haven,Connecticut,9009.0,47375,1529.0,2020-12-31,-8 days,483.0
885129,2021-01-01,New Haven,Connecticut,9009.0,47375,1529.0,2021-01-01,-7 days,0.0
888374,2021-01-02,New Haven,Connecticut,9009.0,48507,1559.0,2021-01-02,-6 days,1132.0
891621,2021-01-03,New Haven,Connecticut,9009.0,48507,1559.0,2021-01-03,-5 days,0.0


In [55]:
## select just last 14 days now that we have daily new cases with 15th day as baseline
covid_last14 = covid_last15[(covid_last15['dt'] > date_N_days_ago) & (covid_last15['dt'] <= data_date_dt)].copy()


In [56]:
## group by FIPS to get case load and follow up demand values for each county
covid_last14_stats = covid_last14.groupby(['fips'])['new_cases'].sum().reset_index(name ='total_cases')

covid_last14_stats[covid_last14_stats.fips == 9009].head(5)

,fips,total_cases
310,9009.0,7432.0


In [57]:
covid.tail(50)

,date,county,state,fips,cases,deaths,dt
262,11/17/2020,Richmond,New York,36085.0,190,2.0,2020-11-17
263,11/18/2020,Richmond,New York,36085.0,197,2.0,2020-11-18
264,11/19/2020,Richmond,New York,36085.0,200,0.0,2020-11-19
265,11/20/2020,Richmond,New York,36085.0,222,1.0,2020-11-20
266,11/21/2020,Richmond,New York,36085.0,161,4.0,2020-11-21
267,11/22/2020,Richmond,New York,36085.0,177,1.0,2020-11-22
268,11/23/2020,Richmond,New York,36085.0,289,1.0,2020-11-23
269,11/24/2020,Richmond,New York,36085.0,237,3.0,2020-11-24
270,11/25/2020,Richmond,New York,36085.0,293,4.0,2020-11-25
271,11/26/2020,Richmond,New York,36085.0,90,2.0,2020-11-26


In [58]:
## group by FIPS to get case load and follow up demand values for each county
#covid_death = covid.groupby(['fips'])['deaths'].sum().reset_index(name ='total_deaths')
#covid_death.head(250)

In [59]:
## group by FIPS to get case load each month and follow up demand values for each county
covid['month'] = pd.to_datetime(covid['date'])
covid['year'] = pd.to_datetime(covid['date'])


In [60]:
#covid_last14_stats_montly = covid.groupby(['fips', covid.month.dt.month, covid.year.dt.year])['deaths'].sum().reset_index() 

#covid_last14_stats_montly['cumulative_death'] = covid_last14_stats_montly.groupby(['fips'])['deaths'].cumsum(axis = 0) 
#covid_last14_stats_montly = covid_last14_stats_montly.groupby(['fips', covid_last14_stats_montly.month,  covid_last14_stats_montly.year])['deaths'].cumsum()

#covid_last14_stats_montly.head(10)

In [61]:
# adding population information from CDC svi dataset
covid_last14_stats = covid_last14_stats.reset_index()
covid_last14_stats['fips'] = covid_last14_stats['fips'].astype(int)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
covid_last14_stats = pd.merge(left = covid_last14_stats, right = svi_county[['E_TOTPOP','FIPS', 'STATE']], how = 'right', right_on = 'FIPS', left_on = 'fips' )
covid_last14_stats.fillna(0 , inplace=True)

In [62]:

covid_last14_stats.tail(5)

,index,fips,total_cases,E_TOTPOP,FIPS,STATE
3215,3217.0,72149.0,29.0,22993,72149,PUERTO RICO
3216,3218.0,72151.0,99.0,34149,72151,PUERTO RICO
3217,3219.0,72153.0,46.0,36439,72153,PUERTO RICO
3218,0.0,0.0,0.0,2132,2105,ALASKA
3219,0.0,0.0,0.0,689,2282,ALASKA


In [63]:
now = pd.to_datetime("now")

print (now)
m_now = now.month
print (m_now)

y_now = now.year

2021-01-08 23:51:43.934571
1


In [64]:
#one_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-1) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#one_month_lag_death = dict(zip(one_month_lag_cumulative_death.fips, one_month_lag_cumulative_death.cumulative_death))

#one_month_lag_cumulative_death.head(5)


In [65]:
#two_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-2) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#two_month_lag_death = dict(zip(two_month_lag_cumulative_death.fips, two_month_lag_cumulative_death.cumulative_death))


#two_month_lag_cumulative_death.head(5)

In [66]:
#three_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now - 3) & (covid_last14_stats_montly['year'] == y_now)]


# Create a dictionary for the 3 last month COVID deaths in each county
#three_month_lag_death = dict(zip(three_month_lag_cumulative_death.fips, three_month_lag_cumulative_death.cumulative_death))

#three_month_lag_cumulative_death.head(5)

#print (three_month_lag_death)

In [67]:

# Create a dictionary for the states of the given the county FIPS
county_of_states = dict(zip(svi_county.FIPS, svi_county.STATE))

# Create a dictionary for the name of the given the county FIPS
county_name = dict(zip(svi_county.FIPS, svi_county.COUNTY))

# Create the list for county FIPS, we consider counties as analogy to the center for community health workers
location = svi_county.FIPS.tolist() #[k for k in SVI_county] #[9001, 9003, 9005, 9007, 9009, 9011, 9013, 9015]#[k for k in SVI_county]




In [68]:
for j in County_covid_death:
       
    print (j, County_covid_death[j])

1001.0 50.0
1003.0 171.0
1005.0 35.0
1007.0 48.0
1009.0 72.0
1011.0 22.0
1013.0 46.0
1015.0 158.0
1017.0 63.0
1019.0 25.0
1021.0 54.0
1023.0 22.0
1025.0 26.0
1027.0 34.0
1029.0 17.0
1031.0 42.0
1033.0 62.0
1035.0 17.0
1037.0 7.0
1039.0 43.0
1041.0 36.0
1043.0 78.0
1045.0 64.0
1047.0 45.0
1049.0 62.0
1051.0 85.0
1053.0 34.0
1055.0 105.0
1057.0 21.0
1059.0 39.0
1061.0 23.0
1063.0 20.0
1065.0 34.0
1067.0 8.0
1069.0 70.0
1071.0 34.0
1073.0 748.0
1075.0 16.0
1077.0 71.0
1079.0 44.0
1081.0 79.0
1083.0 60.0
1085.0 34.0
1087.0 35.0
1089.0 179.0
1091.0 27.0
1093.0 46.0
1095.0 92.0
1097.0 518.0
1099.0 12.0
1101.0 282.0
1103.0 103.0
1105.0 19.0
1107.0 31.0
1109.0 23.0
1111.0 31.0
1113.0 7.0
1117.0 105.0
1115.0 74.0
1119.0 24.0
1121.0 76.0
1123.0 100.0
1125.0 222.0
1127.0 170.0
1129.0 26.0
1131.0 20.0
1133.0 34.0
2013.0 0.0
2016.0 0.0
2020.0 124.0
2050.0 12.0
2060.0 0.0
2068.0 0.0
2070.0 1.0
2090.0 20.0
2100.0 0.0
2110.0 3.0
2122.0 16.0
2130.0 1.0
2150.0 2.0
2158.0 3.0
2164.0 0.0
2170.0 18.0
2180.

26141.0 18.0
26143.0 27.0
26145.0 423.0
26151.0 67.0
26153.0 4.0
26155.0 59.0
26147.0 181.0
26149.0 60.0
26157.0 101.0
nan 43.0
26159.0 83.0
26161.0 215.0
26163.0 3742.0
26165.0 20.0
27001.0 33.0
27003.0 339.0
27005.0 37.0
27007.0 44.0
27009.0 83.0
27011.0 3.0
27013.0 32.0
27015.0 33.0
27017.0 38.0
27019.0 34.0
27021.0 20.0
27023.0 31.0
27025.0 28.0
27027.0 78.0
27029.0 14.0
27031.0 0.0
27033.0 16.0
27035.0 68.0
27037.0 291.0
27039.0 4.0
27041.0 64.0
27043.0 11.0
27045.0 2.0
27047.0 18.0
27049.0 50.0
27051.0 7.0
27053.0 1426.0
27055.0 13.0
27057.0 37.0
27059.0 35.0
27061.0 41.0
27063.0 9.0
27065.0 18.0
27067.0 67.0
27069.0 18.0
27071.0 9.0
27073.0 15.0
27075.0 15.0
27077.0 1.0
27079.0 15.0
27081.0 1.0
27083.0 29.0
27087.0 7.0
27089.0 15.0
27091.0 25.0
27085.0 41.0
27093.0 33.0
27095.0 45.0
27097.0 43.0
27099.0 26.0
27101.0 5.0
27103.0 35.0
27105.0 46.0
27107.0 8.0
27109.0 65.0
27111.0 54.0
27113.0 14.0
27115.0 12.0
27117.0 22.0
27119.0 52.0
27121.0 5.0
27123.0 710.0
27125.0 4.0
27127.0

51550.0 106.0
51041.0 178.0
51043.0 5.0
51570.0 29.0
51580.0 4.0
51045.0 2.0
51047.0 23.0
51049.0 4.0
51590.0 63.0
51051.0 6.0
51053.0 16.0
51595.0 30.0
51057.0 2.0
51059.0 697.0
51600.0 10.0
51610.0 6.0
51061.0 29.0
51063.0 17.0
51065.0 11.0
51067.0 25.0
51620.0 18.0
51069.0 33.0
51630.0 11.0
51640.0 36.0
51071.0 4.0
51073.0 8.0
51075.0 7.0
51077.0 25.0
51079.0 4.0
51081.0 17.0
51083.0 42.0
51650.0 51.0
51085.0 78.0
51660.0 46.0
51087.0 298.0
51089.0 58.0
51091.0 0.0
51670.0 11.0
51093.0 29.0
51095.0 31.0
51099.0 10.0
51101.0 4.0
51097.0 2.0
51103.0 3.0
51105.0 32.0
51678.0 5.0
51107.0 162.0
51109.0 11.0
51111.0 3.0
51680.0 56.0
51113.0 4.0
51685.0 8.0
51683.0 29.0
51690.0 27.0
51115.0 4.0
51117.0 39.0
51119.0 12.0
51121.0 45.0
51125.0 4.0
51127.0 5.0
51700.0 72.0
51710.0 113.0
51131.0 31.0
51133.0 10.0
51720.0 1.0
51135.0 25.0
51137.0 14.0
51139.0 34.0
51141.0 28.0
51730.0 33.0
51143.0 41.0
51735.0 6.0
51740.0 87.0
51145.0 6.0
51147.0 18.0
51149.0 9.0
51153.0 264.0
51155.0 36.0
51750

In [69]:
K_c.head()

,fips,cases,deaths
0,36047.0,12180,162


In [70]:
K = dict(zip(K_c.fips, K_c.cases))
Q = dict(zip(Q_c.fips, Q_c.cases))
B = dict(zip(B_c.fips, B_c.cases))
M = dict(zip(M_c.fips, M_c.cases))
R  = dict(zip(R_c.fips, R_c.cases))

In [71]:
print (K)

{36047.0: 12180}


In [72]:
# Create a dictionary for the county and covid cases
covid_cases_county_ny_times = dict(zip(covid_last14_stats.fips, covid_last14_stats.total_cases))
COVID_14days = {}

for j in location:
    if j in covid_cases_county_ny_times:
        COVID_14days[j] = covid_cases_county_ny_times[j] 
        print(j, covid_cases_county_ny_times[j])
        

        
    else:
        COVID_14days[j] = 0
        
        
    if j not in County_covid_death:
        print ('j',j)
        County_covid_death[j] = 0

#print (three_month_lag_death)
#print (two_month_lag_covid_death)
#print (one_month_lag_covid_death)
#for j in location:
#    if j not in three_month_lag_death:        
#        three_month_lag_death[j] = 0
        
#    if j not in two_month_lag_death:        
#        two_month_lag_death[j] = 0
        
#    if j not in one_month_lag_death:        
#        one_month_lag_death[j] = 0
        
    #if j not in County_covid_death:
    #    County_covid_death[j] = 0

35039 410.0
1001 715.0
1009 516.0
1013 152.0
1015 1116.0
1017 352.0
1031 548.0
1033 516.0
1039 283.0
1043 831.0
1045 427.0
1051 1222.0
1055 1159.0
1067 227.0
1069 1054.0
1071 594.0
1077 891.0
1079 296.0
1083 878.0
1089 3480.0
1095 796.0
1097 3130.0
1103 1250.0
1111 109.0
1113 320.0
1115 819.0
1117 2195.0
1121 710.0
2261 40.0
4021 6008.0
5009 308.0
5011 162.0
5033 703.0
5037 261.0
5045 1550.0
5047 151.0
5051 1151.0
5053 193.0
5063 389.0
5083 234.0
5085 840.0
5087 148.0
5115 830.0
5117 81.0
5121 218.0
5125 1309.0
5131 1590.0
5145 1081.0
6007 1501.0
6017 1355.0
6023 459.0
6027 98.0
6061 2776.0
6079 3093.0
6089 1310.0
6093 204.0
8019 44.0
8039 93.0
8047 11.0
8051 77.0
8057 8.0
8065 56.0
8067 207.0
8071 228.0
8075 158.0
8077 1097.0
8083 142.0
8085 366.0
8093 47.0
8099 74.0
8101 807.0
8103 21.0
8119 91.0
8121 10.0
9005 1113.0
9007 1467.0
9011 2519.0
9015 1413.0
10001 1660.0
12019 2094.0
12033 3584.0
12089 1079.0
12101 3474.0
12109 2694.0
12113 2115.0
13015 1426.0
13039 354.0
13045 756.0
1304

47161 126.0
47163 1429.0
47165 2039.0
47167 602.0
47177 457.0
47179 1405.0
47183 313.0
47185 292.0
47189 1656.0
48009 142.0
48011 9.0
48023 37.0
48031 20.0
48043 28.0
48049 533.0
48053 113.0
48059 163.0
48065 45.0
48067 129.0
48077 167.0
48083 140.0
48091 696.0
48097 502.0
48119 28.0
48143 370.0
48149 100.0
48151 27.0
48159 63.0
48175 10.0
48179 68.0
48181 1317.0
48183 751.0
48187 1240.0
48203 456.0
48221 867.0
48223 223.0
48233 80.0
48235 8.0
48241 323.0
48251 2277.0
48259 272.0
48277 968.0
48281 118.0
48307 25.0
48337 273.0
48343 96.0
48361 645.0
48363 304.0
48367 2002.0
48373 157.0
48381 1078.0
48387 64.0
48395 122.0
48399 193.0
48441 950.0
48459 470.0
48467 284.0
48477 204.0
48503 112.0
49017 60.0
49025 28.0
49055 6.0
50003 157.0
50005 50.0
50011 114.0
50013 7.0
50015 22.0
50017 42.0
50019 15.0
50023 59.0
50025 173.0
50027 151.0
51005 175.0
51007 115.0
51009 324.0
51011 199.0
51015 847.0
51017 69.0
51019 615.0
51023 222.0
51031 464.0
51033 174.0
51035 225.0
51041 1870.0
51043 101.0

26001 37.0
26033 295.0
26035 118.0
26049 1600.0
26069 72.0
26095 11.0
26129 103.0
26153 28.0
27031 9.0
27053 5059.0
27087 15.0
27103 142.0
28005 84.0
28009 80.0
28015 54.0
28023 102.0
28039 130.0
28043 131.0
28071 403.0
28087 702.0
28091 151.0
28107 340.0
28111 102.0
28117 269.0
28145 462.0
28147 96.0
29001 219.0
29015 125.0
29023 181.0
29029 275.0
29063 76.0
29149 75.0
29185 55.0
29197 26.0
29199 8.0
29229 158.0
30023 30.0
30041 132.0
30053 132.0
30061 31.0
30101 12.0
31005 1.0
31133 17.0
31153 1377.0
32013 89.0
32017 36.0
32029 2.0
32031 3242.0
34003 9772.0
35001 4282.0
35015 679.0
35017 206.0
35021 0.0
35035 236.0
35037 21.0
35049 795.0
35061 456.0
36001 3136.0
36041 27.0
36067 4609.0
37041 125.0
37051 2356.0
37075 99.0
37119 10254.0
37131 155.0
37133 1854.0
37147 2094.0
38105 135.0
39009 369.0
39035 9568.0
39061 6941.0
39105 150.0
39121 129.0
39145 792.0
39163 103.0
40005 168.0
40023 191.0
40059 18.0
40067 106.0
40089 358.0
40095 190.0
40141 62.0
41011 155.0
41021 8.0
41023 6.0
410

21189 48.0
21193 209.0
22027 214.0
22039 273.0
28001 124.0
28011 375.0
28027 152.0
28063 29.0
28079 235.0
28083 271.0
28119 37.0
28143 66.0
29155 53.0
30003 145.0
37155 1618.0
38085 24.0
42101 8004.0
45067 389.0
46007 11.0
46041 78.0
46071 7.0
48109 8.0
48141 5333.0
48163 252.0
48261 3.0
51595 61.0
54059 257.0
1005 218.0
1063 93.0
1105 83.0
2050 622.0
4001 895.0
5077 98.0
6019 9969.0
6107 5387.0
12047 131.0
13193 107.0
13239 20.0
13259 25.0
21119 79.0
21147 211.0
21159 91.0
21237 45.0
22035 69.0
28051 79.0
28053 61.0
28103 129.0
28133 224.0
28163 185.0
30085 31.0
35006 208.0
35029 151.0
46017 3.0
46031 8.0
47095 117.0
48061 2094.0
48215 4114.0
48377 38.0
48445 139.0
48479 2172.0
48489 136.0
53001 80.0
54047 161.0
1085 98.0
2158 220.0
6025 1855.0
12049 193.0
12051 344.0
13037 54.0
13265 19.0
22065 155.0
28055 8.0
28123 193.0
28135 109.0
28157 31.0
35031 888.0
36005 1939.0
37079 143.0
45069 277.0
46121 43.0
46137 19.0
48047 59.0
48229 30.0
48323 883.0
48427 870.0
28021 59.0
45005 48.0
46

In [73]:
for j in location: 
    if j in K:
        COVID_14days[j] = K[j]
        print (j, COVID_14days[j], K[j])

    if j in Q:
        COVID_14days[j] = Q[j]
        print (j, COVID_14days[j])
        
    if j in B:
        COVID_14days[j] = B[j]
        print (j, COVID_14days[j])
        
    if j in M:
        COVID_14days[j] = M[j]
        print (j, COVID_14days[j])

    if j in R:
        COVID_14days[j] = R[j]
        print (j, COVID_14days[j])

36085 4369
36061 5478
36047 12180 12180
36081 12561
36005 7649


In [74]:
print (covid_cases_county_ny_times)

{1001.0: 715.0, 1003.0: 2179.0, 1005.0: 218.0, 1007.0: 256.0, 1009.0: 516.0, 1011.0: 150.0, 1013.0: 152.0, 1015.0: 1116.0, 1017.0: 352.0, 1019.0: 128.0, 1021.0: 358.0, 1023.0: 12.0, 1025.0: 277.0, 1027.0: 98.0, 1029.0: 98.0, 1031.0: 548.0, 1033.0: 516.0, 1035.0: 60.0, 1037.0: 86.0, 1039.0: 283.0, 1041.0: 185.0, 1043.0: 831.0, 1045.0: 427.0, 1047.0: 213.0, 1049.0: 527.0, 1051.0: 1222.0, 1053.0: 341.0, 1055.0: 1159.0, 1057.0: 143.0, 1059.0: 289.0, 1061.0: 238.0, 1063.0: 93.0, 1065.0: 219.0, 1067.0: 227.0, 1069.0: 1054.0, 1071.0: 594.0, 1073.0: 7698.0, 1075.0: 107.0, 1077.0: 891.0, 1079.0: 296.0, 1081.0: 1400.0, 1083.0: 878.0, 1085.0: 98.0, 1087.0: 102.0, 1089.0: 3480.0, 1091.0: 158.0, 1093.0: 262.0, 1095.0: 796.0, 1097.0: 3130.0, 1099.0: 237.0, 1101.0: 2056.0, 1103.0: 1250.0, 1105.0: 83.0, 1107.0: 189.0, 1109.0: 322.0, 1111.0: 109.0, 1113.0: 320.0, 1115.0: 819.0, 1117.0: 2195.0, 1119.0: 67.0, 1121.0: 710.0, 1123.0: 221.0, 1125.0: 2095.0, 1127.0: 543.0, 1129.0: 138.0, 1131.0: 78.0, 1133.0

In [75]:
#Parameters
pro_c_s = [(i,county_of_states[i]) for i in location ]
cartesian_pro_county_state = gp.tuplelist(pro_c_s)


In [76]:
df = covid_last14_stats
df['fips'] = df['fips'].astype(int)

In [77]:
###############################################################################################
######################## END calculating different types of vulnerabilities ###################

Since we allocate CHW proportional to the county values of certain vulnaribilites within state, we need a few function to help us with the calculations. 



In [78]:
# This function return the value for the state for the given dictionary

# More specifically sum upt the values for the counties of each state

def total_state(dict_1):
    state_dict = {}
    for s in State:
        state_dict [s] = sum(float(dict_1[j]) for (j,s) in cartesian_pro_county_state.select('*', s) if j in dict_1)  
    return state_dict



In [79]:
# Calculte the population per state by summing up the population in each county in the state
State_pop = total_state(population_county)

In [80]:


# This function returns the ratio of the dict value for county and state of the county
def Proportional(county_level, state_level):
    
    prop = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            prop[j] = (float(county_level[j])/float(state_level[s]))    
        else:
            prop[j] = 0
                
    return prop

In [81]:
print (ACI_total)
print (ACI_total[35039])

{1001: 9804, 1003: 33019, 1005: 7332, 1007: 4860, 1009: 8954, 1011: 3100, 1013: 5004, 1015: 25387, 1017: 7391, 1019: 5162, 1021: 9558, 1023: 3433, 1025: 6557, 1027: 3375, 1029: 2964, 1031: 9652, 1033: 9471, 1035: 2710, 1037: 2234, 1039: 6987, 1041: 3339, 1043: 15529, 1045: 10717, 1047: 13248, 1049: 17983, 1051: 12740, 1053: 6301, 1055: 21800, 1057: 4277, 1059: 6775, 1061: 6484, 1063: 2858, 1065: 4977, 1067: 3485, 1069: 23013, 1071: 11372, 1073: 125514, 1075: 3015, 1077: 15659, 1079: 6971, 1081: 23354, 1083: 15392, 1085: 3457, 1087: 5200, 1089: 48472, 1091: 4803, 1093: 6723, 1095: 21817, 1097: 93203, 1099: 5664, 1101: 52425, 1103: 21849, 1105: 3094, 1107: 4752, 1109: 6901, 1111: 4507, 1113: 13627, 1115: 13737, 1117: 19953, 1119: 4144, 1121: 19136, 1123: 9809, 1125: 34475, 1127: 13645, 1129: 4400, 1131: 4082, 1133: 5781, 2013: 451, 2016: 439, 2020: 51594, 2050: 9063, 2060: 120, 2068: 145, 2070: 2082, 2090: 13016, 2100: 731, 2105: 512, 2110: 4856, 2122: 12129, 2130: 2919, 2150: 2451, 2158

In [82]:
#ACI
ACI_State = total_state(ACI_total)

In [83]:
# Create dicts for the variables of SVI
E_POV = dict(zip(svi_county.FIPS, svi_county.EP_POV))
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.EP_UNEMP))
E_PCI = dict(zip(svi_county.FIPS, svi_county.EP_PCI))
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.EP_NOHSDP))
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.EP_AGE65))
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.EP_AGE17))
E_DISABL = dict(zip(svi_county.FIPS, svi_county.EP_DISABL))
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.EP_SNGPNT))
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.EP_MINRTY))
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.EP_LIMENG))
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.EP_MUNIT))
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.EP_MOBILE))
E_CROWD = dict(zip(svi_county.FIPS, svi_county.EP_CROWD))
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.EP_NOVEH))
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.EP_GROUPQ))


# Calculate the state value for the SVI variables
E_POV_State = total_state(E_POV)
E_UNEMP_State = total_state(E_UNEMP) 
E_PCI_State = total_state(E_PCI)
E_NOHSDP_State = total_state(E_NOHSDP) 
E_AGE65_State = total_state(E_AGE65)
E_AGE17_State = total_state(E_AGE17)
E_DISABL_State = total_state(E_DISABL)
E_SNGPNT_State = total_state(E_SNGPNT)
E_MINRTY_State = total_state(E_MINRTY)
E_LIMENG_State = total_state(E_LIMENG)
E_MUNIT_State = total_state(E_MUNIT)
E_MOBILE_State = total_state(E_MOBILE)
E_CROWD_State = total_state(E_CROWD)
E_NOVEH_State = total_state(E_NOVEH)
E_GROUPQ_State = total_state(E_GROUPQ)



# Calculate the proportinal values for the SVI variables
E_POV_Prop = Proportional(E_POV, E_POV_State )
E_UNEMP_Prop = Proportional(E_UNEMP, E_UNEMP_State ) 
E_PCI_Prop = Proportional(E_PCI, E_PCI_State )
E_NOHSDP_Prop = Proportional(E_NOHSDP, E_NOHSDP_State ) 
E_AGE65_Prop = Proportional(E_AGE65, E_AGE65_State )
E_AGE17_Prop = Proportional(E_AGE17, E_AGE17_State )
E_DISABL_Prop = Proportional(E_DISABL, E_DISABL_State )
E_SNGPNT_Prop = Proportional(E_SNGPNT, E_SNGPNT_State )
E_MINRTY_Prop = Proportional(E_MINRTY, E_MINRTY_State )
E_LIMENG_Prop = Proportional(E_LIMENG, E_LIMENG_State )
E_MUNIT_Prop = Proportional(E_MUNIT, E_MUNIT_State )
E_MOBILE_Prop = Proportional(E_MOBILE, E_MOBILE_State )
E_CROWD_Prop = Proportional(E_CROWD, E_CROWD_State )
E_NOVEH_Prop = Proportional(E_NOVEH, E_NOVEH_State )
E_GROUPQ_Prop = Proportional(E_GROUPQ, E_GROUPQ_State )

ACI_Prop = Proportional(ACI_total, ACI_State)


# SVI calculation 

We calculate the ratio of county value to state value by population for each SVI variables (we use EP-estimate percentage- values in the CDC data set), then we take the average of all 15 SVI variables. 

Let SVI variable set be K, where  

K = { Below Poverty, Unemployed, Income, No High School Diploma, Aged 65 or Older, Aged 17 or Younger, Civilian with a Disability, Single-Parent Households, Minority, Speaks English “Less than Well”, Multi-Unit Structures, Mobile Homes, Crowding, No Vehicle, Group Quarters }

We will use these variables in a county base and state base. While County base values are exactly same as the estimated values for these variables in the CDC website, to calculate the state base, we simply sum the county values for all of the counties in each state for each variable. Then we use the following formula to calculate the SVI value for each county.

Let $S$ is the set of states and $j$ is a county in the state $s$, where $s \in S$, $c^k_j$ SVI variable $k \in K$ value for county j, and $c_s$ SVI variable value for state s.

$SVI_j = \frac{1}{15}\sum_{k \in K} \frac{c^k_j}{c^k_s}$


In [84]:

from collections import Counter
# Sum all SVI variable values for each county
SVI_county_sum = dict(Counter(E_POV_Prop) + Counter(E_UNEMP_Prop) + Counter(E_PCI_Prop) + Counter(E_NOHSDP_Prop) + Counter(E_AGE65_Prop) + Counter(E_AGE17_Prop) + Counter(E_DISABL_Prop) + Counter(E_SNGPNT_Prop) + Counter(E_MINRTY_Prop) + Counter(E_LIMENG_Prop) + Counter(E_MUNIT_Prop) + Counter(E_MOBILE_Prop) + Counter(E_CROWD_Prop) + Counter(E_NOVEH_Prop) + Counter(E_GROUPQ_Prop))

# Divide the sum of all SVI variable values
SVI_county = {j: SVI_county_sum[j]/15 for j in SVI_county_sum }


# Proportional Allocation

We consider allocating 1 million CHW over the states proportional to Medicaid enrollment in each state. Further, we allocate CHW to counties in each state proportional to different county vulnerability criterias as follow.

- MEDICAID
- SVI
- YPLL
- UNEMPLOYMENT
- LAST 14 DAYS COVID CASES
- LAST 14 DAYS COVID CASES / POP
- COVID DEATHS / POP

To calculate the total number of allocated CHW to per county according to these vulnerability criterias, we define the following function called "Proportional_allocation", in which we multiply the CHW allocated to each state with the ratio of the chosen vulnerability criteria of the county to the chosen vulnerability criteria of the state, the function return a dictionary with the counties as keys and the number of CHW allocated to each county for the chosen vulnerability criteria as values. 

In [85]:
def Proportional_allocation(county_level, state_level, state_budget):
    prop_allocate = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            #print (j,s, county_level[j],state_level[s], state_budget[s])
            prop_allocate[j,s] = (float(county_level[j])/float(state_level[s]))*float(state_budget[s])
        
        else:
            prop_allocate[j,s] = 0
            
    
    return prop_allocate

In [86]:
Medicaid_dem = Proportional_allocation(ACI_total, ACI_State, Medicaid_state)
Medicaid_demand = {m[0]: Medicaid_dem[m] for m in Medicaid_dem}
for s in State:
#    print (s)
#    print ( ACI_State[s])
    print (s,Medicaid_state[s])
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

NEW MEXICO 772102
ALABAMA 957116
ALASKA 231145
ARIZONA 1839932
ARKANSAS 830467
CALIFORNIA 11847711
COLORADO 1337805
CONNECTICUT 874974
DELAWARE 239009
FLORIDA 3892552
GEORGIA 1928703
IDAHO 340742
ILLINOIS 2987496
INDIANA 1602976
IOWA 699741
KANSAS 401103
KENTUCKY 1416013
LOUISIANA 1585024
MAINE 232455
MARYLAND 1372695
MASSACHUSETTS 1616404
MICHIGAN 2439425
MINNESOTA 1085778
MISSISSIPPI 632427
MISSOURI 923641
MONTANA 257006
NEBRASKA 254159
NEVADA 685073
NEW HAMPSHIRE 193436
NEW JERSEY 1759653
NEW YORK 6263164
NORTH CAROLINA 1851558
NORTH DAKOTA 96757
OHIO 2788134
OKLAHOMA 797220
OREGON 1053931
PENNSYLVANIA 3069309
RHODE ISLAND 305208
SOUTH CAROLINA 1048276
SOUTH DAKOTA 114059
TENNESSEE 1489536
TEXAS 4457644
UTAH 338812
VERMONT 161049
VIRGINIA 1497770
WASHINGTON 1780968
WEST VIRGINIA 521290
WISCONSIN 1112844
WYOMING 59302
HAWAII 351337
DISTRICT OF COLUMBIA 248591
PUERTO RICO 1622194


In [87]:
#print ('Med', Medicaid_demand[35039])
#print ('Med', Medicaid_demand[4017], 'Pop', population_county[4017], 'Med_capita', 100000*(Medicaid_demand[4017]/population_county[4017]))

In [88]:
#Further we create additional vulnerability values by considering SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita and the number of Medicaid enrolles in each county together

Covid_capita = {j: 100000*(COVID_14days[j]/population_county[j]) for j in location}

Covid_death_capita = {j: 100000*(County_covid_death[j]/population_county[j]) for j in location}

Medicaid_capita = {j: 100000*(Medicaid_demand[j]/population_county[j]) for j in location}

Unemployment_capita = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Rate))

#Three_month_lag_covid_death_capita = {j: 100000*(three_month_lag_death[j]/population_county[j]) for j in location}

#Two_month_lag_covid_death_capita = {j: 100000*(two_month_lag_death[j]/population_county[j]) for j in location}

#One_month_lag_covid_death_capita = {j: 100000*(one_month_lag_death[j]/population_county[j]) for j in location}

In [89]:
for j in location:
    print (j, County_covid_death[j], population_county[j], Covid_death_capita[j])

35039 43.0 38921 110.48020348911898
1001 50.0 55869 89.49506882170793
1009 72.0 57826 124.51146543077509
1013 46.0 19448 236.5281777046483
1015 158.0 113605 139.07838563443508
1017 63.0 33254 189.45089312563903
1031 42.0 52342 80.241488670666
1033 62.0 55241 112.23547727231586
1039 43.0 37049 116.0625118086858
1043 78.0 83768 93.11431572915671
1045 64.0 49172 130.1553729764907
1051 85.0 81209 104.66820180029308
1055 105.0 102268 102.67141236750497
1067 8.0 17205 46.498111014240045
1069 70.0 105882 66.11133148221604
1071 34.0 51626 65.85828845930345
1077 71.0 92729 76.56720119919335
1079 44.0 32924 133.64111286599442
1083 60.0 98915 60.658140827983615
1089 179.0 372909 48.00098683593048
1095 92.0 96774 95.06685680038026
1097 518.0 413210 125.35998644756903
1103 103.0 119679 86.06355333851386
1111 31.0 22722 136.43165214329724
1113 7.0 57961 12.077086316661203
1115 74.0 89512 82.67047993565109
1117 105.0 217702 48.23106815738946
1121 76.0 79978 95.02613218635125
2261 2.0 9202 21.73440556

31145 5.0 10724 46.624393882879524
31147 15.0 7865 190.71837253655434
31155 10.0 21578 46.343498007229584
31157 66.0 35618 185.2995676343422
31165 0.0 1166 0.0
31167 8.0 5920 135.13513513513513
31171 0.0 722 0.0
31177 17.0 20729 82.0107096338463
31181 4.0 3487 114.71178663607685
31183 0.0 783 0.0
31185 8.0 13679 58.483807295854966
32001 39.0 24909 156.56991448873902
32019 45.0 57510 78.24726134585289
33001 58.0 61303 94.61200920020227
33005 7.0 76085 9.200236577511992
33007 16.0 31563 50.69226626112853
33009 9.0 89886 10.012682731459849
33013 75.0 151391 49.540593562365004
33015 152.0 309769 49.068822251419604
33017 35.0 130633 26.792617485627673
33019 6.0 43146 13.906271728549575
34005 643.0 445349 144.38114826798758
34015 420.0 291636 144.0151421635189
34019 140.0 124371 112.56643429738443
34029 1434.0 607186 236.17145322849998
34033 115.0 62385 184.3391840987417
34037 224.0 140488 159.44422299413475
34041 184.0 105267 174.7936200328688
35028 0.0 19369 0.0
36017 26.0 47207 55.0765776

12005 219.0 174705 125.35416845539623
12009 535.0 601942 88.87899498622791
12015 248.0 188910 131.27944523847336
12017 282.0 149657 188.4308786090861
12031 767.0 957755 80.0831110252622
12035 48.0 115081 41.70975226145063
12037 4.0 12125 32.98969072164948
12041 25.0 18582 134.53880099020557
12045 29.0 13639 212.62555905858198
12053 287.0 193920 147.99917491749176
12069 331.0 367118 90.16174635948114
12073 177.0 293582 60.28979978336547
12075 20.0 41503 48.189287521383996
12083 496.0 365579 135.67518922038738
12091 240.0 210738 113.88548814167355
12103 1114.0 974996 114.25687900258052
12105 821.0 724777 113.27622151365178
12111 424.0 328297 129.1513477125895
12117 319.0 471826 67.60966966636006
12119 122.0 132420 92.13109802144692
12121 95.0 44417 213.88207217957088
12127 460.0 553284 83.1399425972918
12131 52.0 74071 70.20291342090697
13013 70.0 83240 84.09418548774627
13035 49.0 24936 196.50304780237408
13051 242.0 289430 83.61261790415644
13067 599.0 760141 78.80116978297447
13083 8.

31163 4.0 3001 133.2889036987671
31169 3.0 5003 59.96402158704777
31175 4.0 4158 96.2000962000962
31179 6.0 9385 63.93180607352157
32005 21.0 48905 42.940394642674576
32007 39.0 52778 73.89442570768122
32011 0.0 2029 0.0
32033 3.0 9580 31.315240083507305
32510 74.0 55916 132.34136919665212
33003 4.0 48910 8.178286648947045
33011 464.0 417025 111.26431269108566
34009 143.0 92039 155.36891969708495
34025 1087.0 618795 175.66399211370486
34027 987.0 491845 200.67297624251543
34035 689.0 328934 209.4645126377936
35027 12.0 19572 61.31207847946045
35043 142.0 146748 96.76452149262681
35055 41.0 32723 125.29413562326192
35059 7.0 4059 172.45627001724563
36003 75.0 46091 162.72157254127706
36007 220.0 190488 115.49283944395448
36009 51.0 76117 67.00211516481207
36011 36.0 76576 47.01211867948182
36013 31.0 126903 24.42810650654437
36015 99.0 83456 118.62538343558283
36019 9.0 80485 11.182207864819532
36029 1263.0 918702 137.47657020448415
36033 9.0 50022 17.99208348326736
36043 32.0 61319 52.

39105 14.0 22907 61.11668922163531
39121 33.0 14424 228.78535773710485
39145 51.0 75314 67.7164936134052
39163 13.0 13085 99.35040122277417
40005 3.0 13758 21.805494984736153
40023 6.0 14672 40.89422028353326
40059 3.0 3688 81.34490238611713
40067 4.0 6002 66.64445184938354
40089 56.0 32832 170.5653021442495
40095 6.0 16931 35.43795404878625
40141 9.0 7250 124.13793103448276
41011 13.0 64487 20.159101834478268
41021 1.0 1912 52.30125523012552
41023 1.0 7199 13.890818169190165
41033 28.0 87487 32.00475499217026
41047 219.0 347818 62.963963912160956
41055 0.0 1780 0.0
41059 59.0 77950 75.68954457985889
41069 1.0 1332 75.07507507507508
42003 1118.0 1216045 91.9373871855071
42045 1022.0 566747 180.32737711889078
45001 26.0 24527 106.00562645248094
45013 112.0 192122 58.29629089849158
45029 58.0 37677 153.94006953844521
45081 28.0 20473 136.76549601914718
45085 123.0 106721 115.25379259939469
45087 45.0 27316 164.738614731293
46011 31.0 35077 88.37699917324743
46049 12.0 2299 521.9660722053

47097 31.0 25633 120.93785354816058
48107 21.0 5737 366.0449712393237
48135 238.0 166223 143.18114821655246
48145 19.0 17297 109.84563797190265
48189 131.0 33406 392.14512363048556
48283 14.0 7520 186.17021276595744
49037 34.0 15308 222.1060883198328
51730 33.0 31346 105.27659031455369
53021 86.0 95222 90.31526327949423
53077 331.0 250873 131.93926807587903
55078 10.0 4556 219.49078138718176
1011 22.0 10101 217.8002178002178
1131 20.0 10373 192.80825219319385
2188 0.0 7621 0.0
2290 4.0 5230 76.48183556405354
5107 25.0 17782 140.59160949274548
6029 523.0 900202 58.09807132177
6031 110.0 152940 71.92363018177063
6047 275.0 277680 99.03486027081532
12027 58.0 38001 152.62756243256757
12039 60.0 45660 131.4060446780552
12059 30.0 19617 152.9285823520416
13003 9.0 8165 110.22657685241886
13049 20.0 13392 149.34289127837516
13071 53.0 45600 116.2280701754386
13093 25.0 13390 186.70649738610905
13141 48.0 8457 567.5771550195104
13309 20.0 7855 254.6148949713558
16033 0.0 845 0.0
21025 3.0 126

In [90]:
County_covid_death[9009], population_county[9009], Covid_death_capita[9009]

(1615.0, 854757, 188.94258836137053)

In [91]:
# Dictionary for the total Covid per capita for each state
Total_covid_cap = total_state(Covid_capita) 


# Dictionary for the total Covid death per capita for each state
Total_covid_death_cap = total_state(Covid_death_capita) 


# Dictionary for the total Medicaid per capita for each state
Total_medicaid_cap = total_state(Medicaid_capita)


# Dictionary for the total Unemployment per capita for each state
Total_unemployment_cap = total_state(Unemployment_capita)



In [92]:
# Dictionaries for the different vulnerability criteria values for states

# Dictionary for total Medicaid patient numbers for each state
Medicaid_demand_state = total_state(Medicaid_demand) 

# Dictionary for total positive COVID cases for last 14 days in each state
Covid_state = total_state(COVID_14days) 

# Dictionary for total SVI values for each state
SVI_state = total_state(SVI_county) 

# Dictionary for total YPLL values for each state
YPLL_state = total_state(YPLL) 

# Dictionary for total Unemployment numbers for each state
Unemployment_state = total_state(Unemployment)  

In [93]:
print(Medicaid_demand_state)

{'NEW MEXICO': 772102.0, 'ALABAMA': 957116.0000000001, 'ALASKA': 231145.0, 'ARIZONA': 1839932.0, 'ARKANSAS': 830467.0000000003, 'CALIFORNIA': 11847711.0, 'COLORADO': 1337805.0, 'CONNECTICUT': 874974.0, 'DELAWARE': 239009.0, 'FLORIDA': 3892551.999999999, 'GEORGIA': 1928702.999999999, 'IDAHO': 340742.0000000001, 'ILLINOIS': 2987496.0000000005, 'INDIANA': 1602975.9999999995, 'IOWA': 699741.0, 'KANSAS': 401103.0000000001, 'KENTUCKY': 1416013.0000000002, 'LOUISIANA': 1585023.9999999995, 'MAINE': 232454.99999999994, 'MARYLAND': 1372695.0, 'MASSACHUSETTS': 1616403.9999999998, 'MICHIGAN': 2439425.0000000005, 'MINNESOTA': 1085778.0000000002, 'MISSISSIPPI': 632427.0000000001, 'MISSOURI': 923640.9999999998, 'MONTANA': 257005.99999999994, 'NEBRASKA': 254159.0, 'NEVADA': 685073.0, 'NEW HAMPSHIRE': 193436.0, 'NEW JERSEY': 1759653.0, 'NEW YORK': 6263164.000000002, 'NORTH CAROLINA': 1851557.999999999, 'NORTH DAKOTA': 96756.99999999999, 'OHIO': 2788133.9999999995, 'OKLAHOMA': 797219.9999999998, 'OREGON

# 1 million CHW allocation to states

We allocate 1 million CHWs to states proportional to total Medicaid enrolles in each state.

Let's $FedCHW$ represents the number of CHW will be allocated within states by the federal government, which is 1 million in our project. $TotMed$ represents the total Medicaid enrollee numbers over the US, $Med_s$ is the total Medicaid enrollee numbers in state $s \in S$, and $CHW_s$ is the total number of CHW allocated to state $ s\in S$. 

$CHW_s = FedCHW*\frac{Med_s}{TotMed}$



In [94]:
# We consider allocation of 1 million CHW all over the US
Federal_budget_CHW = 1000000

# First, we calculate the Total Medicaid enrolles all over the US
Total_federal_need = sum(Medicaid_demand_state[s] for s in State)

# Allocate the 1 million CHWs proportional to Medicaid enrolles in each state
Medicaid_budget_state = {s: (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW  for s in State}

In [95]:
for s in State:
    print (s, Medicaid_demand_state[s], Total_federal_need, Federal_budget_CHW, (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW)

NEW MEXICO 772102.0 76265716.0 1000000 10123.841228999934
ALABAMA 957116.0000000001 76265716.0 1000000 12549.754335224496
ALASKA 231145.0 76265716.0 1000000 3030.7851564653242
ARIZONA 1839932.0 76265716.0 1000000 24125.283239981647
ARKANSAS 830467.0000000003 76265716.0 1000000 10889.126117953188
CALIFORNIA 11847711.0 76265716.0 1000000 155347.79742971272
COLORADO 1337805.0 76265716.0 1000000 17541.368129291543
CONNECTICUT 874974.0 76265716.0 1000000 11472.70419647014
DELAWARE 239009.0 76265716.0 1000000 3133.8983298865246
FLORIDA 3892551.999999999 76265716.0 1000000 51039.342500895145
GEORGIA 1928702.999999999 76265716.0 1000000 25289.25316848791
IDAHO 340742.0000000001 76265716.0 1000000 4467.8266706366485
ILLINOIS 2987496.0000000005 76265716.0 1000000 39172.20156957552
INDIANA 1602975.9999999995 76265716.0 1000000 21018.30395193562
IOWA 699741.0 76265716.0 1000000 9175.040066495934
KANSAS 401103.0000000001 76265716.0 1000000 5259.283214491818
KENTUCKY 1416013.0000000002 76265716.0 10

In [96]:
print (Covid_state)

{'NEW MEXICO': 15187.0, 'ALABAMA': 46838.0, 'ALASKA': 4074.0, 'ARIZONA': 101390.0, 'ARKANSAS': 32651.0, 'CALIFORNIA': 524349.0, 'COLORADO': 31699.0, 'CONNECTICUT': 29904.0, 'DELAWARE': 9270.0, 'FLORIDA': 182068.0, 'GEORGIA': 96020.0, 'IDAHO': 12125.0, 'ILLINOIS': 78501.0, 'INDIANA': 58412.0, 'IOWA': 18723.0, 'KANSAS': 27487.0, 'KENTUCKY': 41165.0, 'LOUISIANA': 41611.0, 'MAINE': 6084.0, 'MARYLAND': 32724.0, 'MASSACHUSETTS': 64652.0, 'MICHIGAN': 43778.0, 'MINNESOTA': 25173.0, 'MISSISSIPPI': 24057.0, 'MISSOURI': 36886.0, 'MONTANA': 5936.0, 'NEBRASKA': 12567.0, 'NEVADA': 27085.0, 'NEW HAMPSHIRE': 10022.0, 'NEW JERSEY': 107765.0, 'NEW YORK': 153786.0, 'NORTH CAROLINA': 97309.0, 'NORTH DAKOTA': 2970.0, 'OHIO': 99418.0, 'OKLAHOMA': 42505.0, 'OREGON': 13258.0, 'PENNSYLVANIA': 96638.0, 'RHODE ISLAND': 10828.0, 'SOUTH CAROLINA': 53084.0, 'SOUTH DAKOTA': 5593.0, 'TENNESSEE': 75394.0, 'TEXAS': 249840.0, 'UTAH': 36945.0, 'VERMONT': 1624.0, 'VIRGINIA': 54773.0, 'WASHINGTON': 31362.0, 'WEST VIRGINIA'

# Proportional allocation for different vulnerability values

Let V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}. We assume $v_j$ represent the vulnerability value for county $j \in J$, while $v_s$ represent the sum of the vulnerability values for each county in the state of county j.

$Prop_{v_j} = \frac{v_j}{v_s}*CHW_s$



In [97]:
# Calling proportional allocation function for different vulnerability criterias

# Proportional allocation according to cumulative Covid death in per capita in each county
Proportional_to_covid_death_cap = Proportional_allocation(Covid_death_capita, Total_covid_death_cap,Medicaid_budget_state)

# Propportional allocation according to Medicaid enrollee number in each county
Proportional_to_medicaid = Proportional_allocation(Medicaid_demand, Medicaid_demand_state,Medicaid_budget_state )

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_medicaid_cap = Proportional_allocation(Medicaid_capita, Total_medicaid_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases in each county
Proportional_to_covid = Proportional_allocation(COVID_14days, Covid_state, Medicaid_budget_state)

# Proportional allocation according to SVI score in each county
Proportional_to_SVI = Proportional_allocation(SVI_county, SVI_state, Medicaid_budget_state)

# Proportional allocation according to YPLL in each county
Proportional_to_YPLL = Proportional_allocation(YPLL, YPLL_state, Medicaid_budget_state)

# Proportional allocation according to Unemployment  in each county
Proportional_to_unemployment = Proportional_allocation(Unemployment, Unemployment_state, Medicaid_budget_state)

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_unemployment_cap = Proportional_allocation(Unemployment_capita, Total_unemployment_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases per capita in each county
Proportional_to_covid_capita = Proportional_allocation(Covid_capita, Total_covid_cap, Medicaid_budget_state)



# Normalize values for comparison
To be able compare the different vulnerability values for each county, we normalize all vulnerability values as follows. 

Let $m_{v_s} = \min \{v_j, \text{ for county j in state s }\}$  and 
$M_{v_s} = \max \{v_j, \text{ for county j in state s }\}$.

We calculate the normalize value for each vulnerability for each county by substracting the min vulnerability in the state of the county and dividing that by the differences between max and min value of the vulnerability values in the state. Mathematical formulation for the normalization is as follows.

$N_{v_j} = \frac{v_j - m_{v_s}}{M_{v_s} - m_{v_s}}$ 

for each $v \in V$, where V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}, j is a county in each state $s \in S$.


In [98]:
#Normalize function to normalize the vulnerability values to be able to compare them

def normalize(dict_1):
    
    result = {}
    min_data = {s: min(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    max_data = {s: max(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    
    for (j,s) in cartesian_pro_county_state:
        
        if (max_data[s] - min_data[s]) != 0 :
    
            result[j] = (dict_1[j] - min_data[s])/(max_data[s] - min_data[s])
        
        else:
            result[j] = 1
    
    return result 

# Percentile Rank

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html"> The function scipy.stats.percentileofscore (a, score, kind='rank')   </a>
computes the percentile rank of a score relative to a list of scores. 
"rank": Average percentage ranking of score. In case of multiple matches, average the percentage rankings of all matching scores.

In [99]:
from scipy import stats

# Calculate percentile ranks

def percentile_ranks(data):
    x = {s: [] for s in State}

    for (j,s) in cartesian_pro_county_state:
         
        x[s].append(data[j])
    
    
    
    percentile_ranks = {i: stats.percentileofscore(x[s], data[i], 'rank') for (i,s) in cartesian_pro_county_state}

    return percentile_ranks

In [100]:
# Write timestamp 

time_stamp = time.strftime('%m-%d-%Y %H:%M:%S')
with open('Output/time_stamp.csv','w') as f:
    w = csv.writer(f)
    now = time.strftime('%m/%d/%Y %H:%M:%S')
    w.writerow(['time',now])
    

In [101]:
#print (Medicaid_demand)

In [102]:
#Write a function to order the dicts
def order_k(dict_1):
    dict_2 = {}
    for m in location:
        if m in dict_1.keys():
            dict_2[m] = dict_1[m]
        else:
            dict_2[m] = 0
    
    return dict_2
            

In [103]:


Medicaid_demand = order_k(Medicaid_demand)#{m: Medicaid_demand[m] for m in location}
COVID_14days = order_k(COVID_14days)#{m: COVID_14days[m] for m in location}
SVI_county = order_k(SVI_county)#{m: SVI_county[m] for m in location}
YPLL = order_k(YPLL)#{m: YPLL[m] for m in location}
Unemployment = order_k(Unemployment)#{m: Unemployment[m] for m in location}
Unemployment_capita = order_k(Unemployment_capita)#{m: Unemployment_capita[m] for m in location}

In [104]:
# Write file allocation with each strategies for each county 

Strategies = ["Medicaid_demand", "Medicaid_capita", "Covid", "SVI"
              , "YPLL","Unemployment", "Unemployment_capita", "Covid_capita",  "Covid_death_capita" ]

fieldnames = []  
fieldnames.append('County_FIPS')


SVI_values = {i:SVI_county[i] for i in location}
s_count = 1
for s in Strategies:   
    fieldnames.append('Proportional_allocation_to_' + s)
    fieldnames.append(s)
    fieldnames.append('Percentile_ranks_' + s)

    
        

writefile = 'Output/County_level_proportional_allocation_for_all_policies.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(fieldnames)
    for row in zip(location
                   , Proportional_to_medicaid.values(),         Medicaid_demand.values(),      percentile_ranks(Medicaid_demand).values()
                   , Proportional_to_medicaid_cap.values(),     Medicaid_capita.values(),      percentile_ranks(Medicaid_capita).values()
                   , Proportional_to_covid.values(),            COVID_14days.values(),         percentile_ranks(COVID_14days).values()
                   , Proportional_to_SVI.values(),              SVI_county.values(),           percentile_ranks(SVI_county).values()
                   , Proportional_to_YPLL.values(),             YPLL.values(),                 percentile_ranks(YPLL).values()
                   , Proportional_to_unemployment.values(),     Unemployment.values(),         percentile_ranks(Unemployment).values()
                   , Proportional_to_unemployment_cap.values(), Unemployment_capita.values(),  percentile_ranks(Unemployment_capita).values()
                   , Proportional_to_covid_capita.values(),     Covid_capita.values(),         percentile_ranks(Covid_capita).values()
                   , Proportional_to_covid_death_cap.values(),  Covid_death_capita.values(),   percentile_ranks(Covid_death_capita).values() ):                    
       
        writer.writerow(row)

In [105]:
writefile = 'Output/State_level_allocation.csv'

cl = ['State', 'CHW_allocation']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(cl)
    for row in zip( State, Medicaid_demand_state.values()):
        writer.writerow(row)